from pkgs.simplifier import simplify
from pkgs.compare import comp

In [168]:
from pkgs import txt
import numpy as np
import itertools

In [169]:
file1 = open('doc1.txt','r')
file2 = open('doc2.txt','r')

a = file1.read()
b = file2.read()

In [170]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
stop = set(stopwords.words("english"))
stop.remove("not")
stop.remove("no")
posIgnore = ["$","''","(",")",",","--",".",":","CD","FW","LS","NNP","NNPS","NNS","PRP","PRP$","SYM","TO","WDT","WP","WP$","WRB","``"]
exStr = "There is a setuid binary in the homedirectory that does the following: it makes a connection to localhost on the port you specify as a commandline argument. It then reads a line of text from the connection and compares it to the password in the previous level."

def getWnetPos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def simplify(parag = exStr):
    stns = sent_tokenize(parag)
    wrds = []
    for stn in stns:
        wrds.append(word_tokenize(stn))
    wrdsFilt = []
    for i in wrds:
        temp = []
        for j in i:
            if j.casefold() not in stop:
                temp.append(j)
        wrdsFilt.append(temp)

    tagged = []
    for words in wrdsFilt:
        tagged.append(nltk.pos_tag(words))
    wrdsLemTag = []
    wrdsLem = []
    for words in tagged:
        temp = []
        for word in words:
            if word[1] in posIgnore:
                temp.append((word[0],word[1]))    
            elif getWnetPos(word[1]) != None:
                temp.append((lemmatizer.lemmatize(word[0],pos=getWnetPos(word[1])),word[1]))
            else:
                temp.append((lemmatizer.lemmatize(word[0]),word[1]))
        wrdsLemTag.append(temp)
    #print(wrdsLemTag)
    for words in wrdsLemTag:
        temp = []
        for word in words:
            temp.append(word[0])
        wrdsLem.append(temp)

    syn = []
    for words in wrdsLem:
        temp = []
        for word in words:
            synons = []
            if word in ['.',',']:
                continue
            if((wrdsLemTag[wrdsLem.index(words)][words.index(word)])[1].startswith('N')):
                temp.append(word)
                continue
            for ss in wordnet.synsets(word):
                synons.append(ss)
            synons.sort()
            if(synons):
                wrd = synons[0]
            synons.clear()
            temp.append(wrd.name().partition('.')[0])
        syn.append(temp)
    return syn

In [178]:
from itertools import product
import nltk
from nltk.corpus import wordnet

def wordSimilarity(wrd1,wrd2):
    syns1 = wordnet.synsets(wrd1)
    syns2 = wordnet.synsets(wrd2)
    for sn1, sn2 in product(syns1, syns2):
        if syns1 == syns2:
            return 1.0
        else:
            return wordnet.wup_similarity(sn1, sn2)


def comp(txt1,txt2):
    assert len(txt1) == len(txt2), "Different number of lines in samples."
    sim = []
    a=txt1.copy()
    b=txt2.copy()

    #This segment creates array of false for each sentence combination
    '''
    for i in txt1:
        tmp=[]
        for j in txt2:
            tmp.append(False)
        sim.append(tmp)
    '''
    sim = np.zeros((len(txt1),len(txt2))) == 1
    #
    #Word Comparasion
    '''
    for i,j in zip(a,b):
        for w1 in i:
            for w2 in j:
                if w1 == w2:
                    sim[a.index(i)][b.index(w1)] = True
                    i.discard(w1)
                    j.discard(w2)
    '''

    for i,j in zip(a,b):
        cross = list(product(i,j))
        app = []
        for a,b in cross:
            if(a==b):
                app.append((a,b))
        for k in range(len(list(cross))):
            for l in range(len(cross[0])):
                sim[k][l] = True

    #
    
    #Scoring
    score = [0]*len(txt1)
    total = score.copy()
    for i in range(len(txt1)):
        for j in range(len(sim[i])):
            total[i][j] += 1
            if sim[i][j]:
                score[i][j] += 1
    #
    
    return (sum(score)/sum(total))*100

In [180]:
#print(simplify(a))
print(comp(simplify(a),simplify(b)))

AttributeError: 'list' object has no attribute 'discard'